In [ ]:
# Turn off Tensorflow warnings

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Key imports

import numpy as np
import tensorflow as tf
import keras as k
import pickle
from math import prod
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

## Helper functions

def load_object(filename):
    with open(filename, 'rb') as imp:  # Overwrites any existing file.
        res = pickle.load(imp)
    return res

In [ ]:
### Load objects 

# cnn_test_data = load_object('test_data.pkl')
# cnn_test_labels = load_object('test_labels.pkl')
# cnn_test_data = np.concatenate([cnn_test_data[d] for d in ['browse','chat','mail','p2p']])
# cnn_test_labels = np.concatenate([cnn_test_labels[d] for d in ['browse','chat','mail','p2p']])

lstm_test_data = load_object('lstm_test_data.pkl')
lstm_test_labels = load_object('lstm_test_labels.pkl')
lstm_test_data = np.concatenate([lstm_test_data[d] for d in ['browse','chat','mail','p2p']])
lstm_test_labels = np.concatenate([lstm_test_labels[d] for d in ['browse','chat','mail','p2p']])
scaler = load_object('scaler.pkl')

# cnn_sae_full = tf.keras.saving.load_model('cnn_sae_full.11.keras')
stacked_cnn_lstm_sae = tf.keras.saving.load_model('stacked_cnn_lstm_sae.53.keras')

In [ ]:
# Calculate confusion matrix for CNN

# y_true = np.argmax(cnn_test_labels, axis=-1) + 1
# y_pred = np.argmax(cnn_sae_full.predict(cnn_test_data), axis=-1) + 1
# mat = confusion_matrix(y_true, y_pred)
# print(mat)


In [ ]:
# Calculate confusion matrix for stacked

y_true = np.argmax(lstm_test_labels, axis=-1) + 1
y_pred = np.argmax(stacked_cnn_lstm_sae.predict(lstm_test_data), axis=-1) + 1
mat = confusion_matrix(y_true, y_pred)
print(mat)

In [ ]:
def skipper(fname):
    with open(fname) as fin:
        no_comments = (line for line in fin if not line.lstrip().startswith('n'))
        next(no_comments, None) # skip header
        for row in no_comments:
            yield row

browse_data = np.loadtxt(skipper('Data/featurePCAP_Browsing.csv'), delimiter=',')
chat_data = np.loadtxt(skipper('Data/featurePCAP_Chat.csv'), delimiter=',')
mail_data = np.loadtxt(skipper('Data/featurePCAP_Mail.csv'), delimiter=',')
p2p_data = np.loadtxt(skipper('Data/featurePCAP_P2P2.csv'), delimiter=',')
raw_data = {'browse': browse_data, 'chat': chat_data, 
            'mail' : mail_data, 'p2p': p2p_data}

t, z = 0, 0
for k in raw_data:
    r = raw_data[k]
    total_count = r.shape[0]
    zero_count = r[np.all(r==0, axis=1)].shape[0]
    t += total_count
    z += zero_count
    print(k, total_count, zero_count, '{:2.1f}'.format(100*zero_count/total_count))
print("total", t, z, '{:2.1f}'.format(100*z/t))